In [1]:
from typing import Union
import os 
os.chdir('..')

import numpy as np
import faiss

In [2]:
from src.elegant_fuzzy_genetic_algorithms.helpers.all_params_wrapper import AllEFGAParamsParallelWrapper
from src.elegant_fuzzy_genetic_algorithms.helpers.cached_indexed_priority_wrapper import CachedPriorityWrapper

In [4]:
def generate_search_space(n_splits: Union[tuple[int], int], ranges: list[tuple[float]]  = [(0, 1), (0, 1)]):
    """Generates space of all combinations of points in range

    Args:
        n_splits (Union[tuple[int], int]): Number of points for each range
        ranges (list[tuple[float]], optional): Ranges. Defaults to [(0, 1), (0, 1)].

    Returns:
        _type_: Search space
    """

    # Making it tuple
    if isinstance(n_splits, int):
        n_splits = (n_splits, n_splits)

    
    range_1 = np.linspace(start=ranges[0][0], stop=ranges[0][1], num=n_splits[0])
    range_2 = np.linspace(start=ranges[1][0], stop=ranges[1][1], num=n_splits[1])

    params_combinations = np.array(np.meshgrid(range_1, range_2)).T.reshape(-1, 2)

    return params_combinations


In [5]:
def init_param_index(params_combinations: np.array) -> faiss.Index:
    """Given combination of parameters creates Index object for faster search

    Args:
        params_combinations (np.array):  Search space

    Returns:
        faiss.Index: Faiss index object
    """
    param_index = faiss.IndexFlatL2(params_combinations.shape[1])
    param_index.add(params_combinations)
    return param_index

In [47]:
params_combinations = generate_search_space(100)
param_index = init_param_index(params_combinations)

In [52]:
# Max number of priorities to be calculated at once

def calculate_priorities(params_combinations, n_terms_params, n_terms_priority, nmax: int = 1000):
    """Calculates priorities given parameter combinations

    Args:
        params_combinations (_type_): _description_
        n_terms_params (_type_): _description_
        n_terms_priority (_type_): _description_
        nmax (int, optional): Max number of entries to be calculated at once. Defaults to 1000.

    Returns:
        _type_: _description_
    """
    n_iterations: int = (params_combinations.shape[0] // nmax) + 1
    priority_inferencer = AllEFGAParamsParallelWrapper(n_terms_params=n_terms_params, n_terms_priority=n_terms_priority)
    
    priorities_final = []
    for i in range(n_iterations):
        priorities_ = priority_inferencer.infer_priority(c1=params_combinations[i*nmax:(i + 1)* nmax, 0], 
                                                         c2=params_combinations[i*nmax:(i + 1)* nmax, 1])
        priorities_final.append(priorities_)
        
    priorities_final  = np.concatenate(priorities_final)
    return priorities_final


In [49]:
y = calculate_priorities(params_combinations, n_terms_params=3, n_terms_priority=7)

In [70]:
np.random.seed(42)
ex1 = np.random.uniform(0, 1, size=(2, 2))
D, I = param_index.search(ex1, k=1)

In [72]:
params_combinations[I]

array([[[0.37373737, 0.94949495]],

       [[0.72727273, 0.5959596 ]]])

In [73]:
ex1

array([[0.37454012, 0.95071431],
       [0.73199394, 0.59865848]])

In [74]:
y[I]

array([[-0.32448131],
       [-0.32610519]])

In [76]:
faiss.write_index(param_index, 'indices/test_ind.index')

In [77]:
test_search = np.random.uniform(0, 1, size=(2000, 2))

In [79]:
%%timeit
D, I = param_index.search(test_search, k=1)
estimations = y[I]

55.9 ms ± 5.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [80]:
priority_inferencer = AllEFGAParamsParallelWrapper(n_terms_params=3, n_terms_priority=7)

In [84]:
%%timeit 
priority_actual = priority_inferencer.infer_priority(test_search[:, 0], test_search[:, 1])

1.57 s ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
priority_actual = priority_inferencer.infer_priority(test_search[:, 0], test_search[:, 1])

In [88]:
D, I = param_index.search(test_search, k=1)
estimations = y[I]

In [96]:
np.median(np.abs(priority_actual -  estimations.ravel()))

0.002955391677721686

~28 times faster in exchange for ~0.003 priority error

In [97]:
def estimate_by_index(index: faiss.Index, y: np.array, query: np.array):
    D, I = index.search(query, k=1)
    estimations = y[I]
    return estimations


In [3]:
cw = CachedPriorityWrapper(7)

In [4]:
cw.infer_priority(np.array([0, .2, .1]), np.array([.98, .27, .1]))

array([[0.0263354 ],
       [0.5214763 ],
       [0.74992265]])